# Проект e-learning: вариант 2

Продакт-менеджер Василий попросил вас проанализировать завершенные уроки и ответить на следующие вопросы:

1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) (7 баллов).

2. Выяви самый сложный и самый простой экзамены: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*. (5 баллов)

3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). (5 баллов) 

4. Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3). (8 баллов)

5. Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов. (10 баллов) 

6. Построй адаптированные под задачу обучения RFM-кластеры студентов. Где R — среднее время сдачи одного экзамена, F — завершаемость курсов, M — среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Примерное описание подхода можно найти тут. (35 баллов)

*завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

Файлы: 

assessments.csv — этот файл содержит информацию об оценках в тестах. Обычно каждый предмет включает ряд оценок, за которыми следует заключительный экзамен:

- code_module — идентификационный код предмета.
- code_presentation — семестр (идентификационный код).
- id_assessment — тест (идентификационный номер ассессмента).
- assessment_type — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).
- date — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).
- weight — вес оценки в %. Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%.

courses.csv — файл содержит список всех доступных модулей (курсов) и их презентации:
- code_module — предмет (идентификационный код)
- code_presentation — семестр (идентификационный код) .
- module_presentation_length — продолжительность семестра в днях.

studentAssessment.csv — этот файл содержит результаты тестов студентов. Если учащийся не сдает (не сдаёт работу, не высылает результат) тест, результат не записывается в таблицу. Заключительные экзамены не принимаются, если результат предварительных тестов отсутствуют в системе:

- id_assessment — тест (идентификационный номер).
- id_student — идентификационный номер студента.
- date_submitted — дата подачи заявки студентом, измеряемая как количество дней с начала семестра.
- is_banked — факт презачета теста с прошлого семестра. 
- score — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 интерпретируется как несдача.

studentRegistration.csv — этот файл содержит информацию о времени, когда студент зарегистрировался для прохождения курса внутри семестра:

- code_module — предмет (идентификационный код)
- code_presentation — семестр (идентификационный код).
- id_student — идентификационный номер студента.
- date_registration — дата регистрации студента, это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).
- date_unregistration — дата отмены регистрации студента с Предмета. У студентов, окончивших курс, это поле остается пустым.

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns 
import datetime as dt 
from operator import attrgetter

### Данные

In [2]:
assessments = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-prohorova-18/file/assessments.csv', encoding='Windows-1251', sep=',')

In [3]:
courses = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-prohorova-18/file/courses.csv', encoding='Windows-1251', sep=',')

In [4]:
student_assessment = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-prohorova-18/file/studentAssessment.csv', encoding='Windows-1251', sep=',')

In [5]:
student_registration = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-prohorova-18/file/studentRegistration.csv', encoding='Windows-1251', sep=',')

In [123]:
assessments.head(2)

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0


In [11]:
assessments.nunique()

code_module            7
code_presentation      4
id_assessment        206
assessment_type        3
date                  74
weight                24
dtype: int64

In [12]:
assessments.isna().sum()

code_module           0
code_presentation     0
id_assessment         0
assessment_type       0
date                 11
weight                0
dtype: int64

In [124]:
courses.head(2)

,code_module,id_student,successful_exam,finalizability
0,CCC,1915,1683,0.88
1,DDD,3044,2709,0.89


In [125]:
student_assessment.head(2)

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0


In [10]:
student_assessment.nunique()

id_assessment       188
id_student        23369
date_submitted      312
is_banked             2
score               101
dtype: int64

In [13]:
student_assessment.isna().sum()

id_assessment       0
id_student          0
date_submitted      0
is_banked           0
score             173
dtype: int64

In [126]:
student_registration.head(2)

,code_module,code_presentation,id_student,date_registration,date_unregistration,cancel
0,AAA,2013J,11391,-159,No,False
1,AAA,2013J,28400,-53,No,False


In [14]:
student_registration.nunique()

code_module                7
code_presentation          4
id_student             28785
date_registration        332
date_unregistration      416
dtype: int64

In [15]:
student_registration.isna().sum()

code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64

---

### 1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) 

Курс - 1 предмет (1 семестр).
Успешная сдача - зачет по курсу на экзамене, т.е. нужно выбрать из датафрейма с оценками student_assessment экзамены и при этом оценка (score) должна быть больше 40+.

In [105]:
# мерджим датасеты с оценками и с описанием тестов
# выбираем типы теста - экзамен и результат больше 39
# группируем по студентам и находим количество тестов
# выбираем только тех студентов, количество сданных экзаменов у которых больше 1

pd.merge(student_assessment, assessments[['id_assessment', 'assessment_type']], on='id_assessment', how='left')\
.query('(assessment_type == "Exam") and (score > 39)')\
.groupby('id_student', as_index=False)\
.agg({'id_assessment': 'count'})\
.query('id_assessment == 1')\
.shape[0]

3802

Таким образом, 3802 учеников сдали только один курс.

----

### 2. Выяви самый сложный и самый простой экзамены: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*

*завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

успешный экзамен - оценка больше 40

In [171]:
pd.merge(student_assessment, assessments, on='id_assessment', how='left')\
.query('(assessment_type == "Exam") and  (is_banked == 0)')\
.assign(successful = lambda x: (x.score > 39).astype(int))\
.groupby(['code_module','id_assessment'] ,as_index=False)\
.agg({'successful':'mean'})\
.round(2)\
.sort_values('successful')

,code_module,id_assessment,successful
2,DDD,25340,0.84
1,CCC,24299,0.87
0,CCC,24290,0.89
5,DDD,25368,0.89
3,DDD,25354,0.91
4,DDD,25361,0.93


In [172]:
pd.merge(student_assessment, assessments, on='id_assessment', how='left')\
.query('(assessment_type == "Exam") and  (is_banked == 0)')\
.assign(successful = lambda x: (x.score > 39).astype(int))\
.groupby(['code_module'] ,as_index=False)\
.agg({'successful':'mean'})\
.round(2)\
.sort_values('successful')

,code_module,successful
0,CCC,0.88
1,DDD,0.89


Самая высокая завершаемость - 0.89 у курса DDD, низкая у CCC - 0.88. 

Самая высокая завершаемость - 0.93 у экзамена 25361	по предмету DDD , самая низкая - 0.84 у экзамена 25340 по предмету DDD.

---

### 3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом).

Дату сдачи считаем по date_submitted. Также необходимо отфильтровать студентов, которым перезачли экзамены.


In [162]:
pd.merge(student_assessment, assessments, on='id_assessment', how='left')\
.query('(assessment_type == "Exam") and (is_banked == 0)')\
.groupby(['code_module'], as_index=False)\
.agg({'date_submitted': 'mean'})\
.round(2)

,code_module,date_submitted
0,CCC,239.41
1,DDD,237.90


Средний срок сдачи экзамена по предмету CCC - 239.41 дней с начала семестра, по предмету DDD - 237.90 дней с начала семестра.

----

### 4. Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3).

date_registration — дата регистрации студента, это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).
date_unregistration — дата отмены регистрации студента с Предмета. У студентов, окончивших курс, это поле остается пустым.

Самые популярные - количество курсов с датой регистрации и при условии что отсутсвует дата отменты регистрации 

Отток - количество отмен регистраций на курс / общее количество регистраций на курс

In [86]:
student_registration = student_registration.fillna("No")

In [90]:
student_registration.query('(date_registration != "No") and (date_unregistration == "No")')\
.groupby('code_module', as_index=False)\
.agg({'date_registration':'count'})\
.sort_values('date_registration', ascending=False)\
.head(3)

,code_module,date_registration
1,BBB,5531
5,FFF,5381
3,DDD,4034


Самые популярные курсы - BBB, FFF, DDD.

In [94]:
student_registration['cancel'] = student_registration.date_unregistration != "No"

In [220]:
student_registration.head(3)

,code_module,code_presentation,id_student,date_registration,date_unregistration,cancel
0,AAA,2013J,11391,-159,No,False
1,AAA,2013J,28400,-53,No,False
2,AAA,2013J,30268,-92,12,True


In [100]:
df4 = student_registration.query('(date_registration != "No")')\
.groupby('code_module', as_index=False)\
.agg({'date_registration': 'count', 'cancel': 'sum'})\
.sort_values('date_registration', ascending=False)

In [101]:
df4['CR'] = (df4.cancel / df4.date_registration).round(2)

In [147]:
df4.sort_values('CR', ascending=False).head(3)

,code_module,date_registration,cancel,CR
2,CCC,4426,1940,0.44
3,DDD,6257,2223,0.36
5,FFF,7751,2370,0.31


Курсы с самым большим оттоком - CCC, DDD, FFF

---

### 5. Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов. 

In [239]:
def semester_analysis(df, semester, date, score, score_boundary):
    # функция для построения семестрового (когортного) анализа
    # df - dataframe (отфильтрованный)
    # semester - колонка с семестром
    # date - колонка с датой сдачи экзамена
    # score - колонка с результатами экзамена
    # score_boundary - пограничный балл (больше - экзамен сдан, меньше - не сдан)
    
    df = df.assign(finalizability = lambda x: (x.score > score_boundary).astype(int))\
    .groupby(semester, as_index=False)\
    .agg({date:'mean', 'finalizability':'mean'})\
    .round(2)
    
    return df

In [342]:
def semester_analysis_visualization(df):
    """ Функция для визуализации семестрового (когортного) анализа
    параметры:
          df - DateFrame            
    """
    final_retention = (df
            .style
            .background_gradient(cmap='YlOrRd')) # раскрашиваем ячейки по столбцам)  
                       
    return final_retention

In [229]:
df = pd.merge(student_assessment, assessments, on='id_assessment', how='left')\
.query('(assessment_type == "Exam") and (is_banked == 0)')

In [321]:
data = semester_analysis(df, 'code_presentation', 'date_submitted', 'score', 39)

In [343]:
semester_analysis_visualization(data)

,code_presentation,date_submitted,finalizability
0,2013B,230.180000,0.840000
1,2013J,239.510000,0.910000
2,2014B,232.990000,0.900000
3,2014J,243.670000,0.880000


Семестр с самой низкой завершаемостью - 2013B, с самым долгим средним сроком сдачи курсов - 2014J

----

### 6. Построй адаптированные под задачу обучения RFM-кластеры студентов. Где R — среднее время сдачи одного экзамена, F — завершаемость курсов, M — среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры.

R — среднее время сдачи одного экзамена, F — завершаемость курсов, M — среднее количество баллов, получаемое за экзамен.
Посчитаем метрики:

In [163]:
date_mean = pd.merge(student_assessment, assessments, on='id_assessment', how='left')\
.query('(assessment_type == "Exam") and (is_banked == 0)')\
.groupby(['id_student'], as_index=False)\
.agg({'date_submitted': 'mean'})\
.round(2)

In [180]:
finalizability = pd.merge(student_assessment, assessments, on='id_assessment', how='left')\
.query('(assessment_type == "Exam") and  (is_banked == 0)')\
.assign(successful = lambda x: (x.score > 39).astype(int))\
.groupby('id_student', as_index=False)\
.agg({'successful':'mean'})\
.round(3)

In [157]:
score_mean = pd.merge(student_assessment, assessments, on='id_assessment', how='left')\
.query('(assessment_type == "Exam") and (is_banked == 0)')\
.groupby(['id_student'], as_index=False)\
.agg({'score': 'mean'})\
.round(2)

In [181]:
metrics = pd.merge(date_mean, finalizability, on='id_student')

In [182]:
metrics = pd.merge(metrics, score_mean, on='id_student')

In [223]:
metrics.describe()[3:]

,id_student,date_submitted,successful,score
min,23698.0,229.0,0.0,0.0
25%,501158.0,234.0,1.0,50.0
50%,588482.0,241.0,1.0,66.0
75%,646351.0,243.0,1.0,82.0
max,2698251.0,285.0,1.0,100.0


In [184]:
metrics['successful'].unique()

array([1. , 0. , 0.5])

Разобьем на сегменты на основании квантилей в случае R и M, и т.к. значения завершаемости курсов - или меньше 1, или равно 1, интервалы для этой метрики выберем самостоятельно (2 сегмента: 1 - < 1, 2 - >=1): 

In [214]:
metrics['R'] = pd.qcut(metrics['date_submitted'], 3, ['3', '2', '1'])
metrics['F'] = pd.cut(metrics['successful'], bins=[0, 0.99, 2], labels=['1', '2'])
metrics['M'] = pd.qcut(metrics['score'], 3, ['1', '2', '3'])

In [219]:
metrics.head(2)

,id_student,date_submitted,successful,score,R,F,M,RFM
0,23698,243.0,1.0,80.0,2,2,3,223
1,24213,236.0,1.0,58.0,3,2,2,322


R: 

     1 - большее среднее время сдачи экзамена (> 243 дней),   
     2 - среднее (234 - 243 дней),  
     3 - меньшее среднее время сдачи экзамена (< 234 дней).
    
F: 

     1 - низкая завершаемость (< 1),   
     3 - высокая (= 1).    
    
M: 

     1 - низкий балл (< 50),  
     2 - средний  (50 - 82),  
     3 - высокий (> 82). 

In [216]:
# Создаем колонку с сегментами
metrics['RFM'] = metrics['R'].astype('str') + metrics['F'].astype('str') + metrics['M'].astype('str')

In [224]:
metrics.head(3)

,id_student,date_submitted,successful,score,R,F,M,RFM
0,23698,243.0,1.0,80.0,2,2,3,223
1,24213,236.0,1.0,58.0,3,2,2,322
2,27116,243.0,1.0,96.0,2,2,3,223
